In [35]:
import os
import pandas as pd
from pathlib import Path
from datasets import load_dataset, DownloadMode, DatasetDict
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from matplotlib import pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping

In [36]:
destination_path = '\Data'

dataset_emotion_split = load_dataset(
    "dair-ai/emotion", "split", 
    cache_dir=destination_path,
    download_mode=DownloadMode.REUSE_DATASET_IF_EXISTS
)

print(dataset_emotion_split)

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\klykk\AppData\Local\Temp\ipykernel_15004\1141551962.py:1: SyntaxWarning: invalid escape sequence '\D'
  destination_path = '\Data' #'RNN\Data'


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})


Train_test_split til unsplit data

In [37]:
"""
# Split the emotion dataset into training, and test sets
hf_train = dataset_emotion_split['train'].train_test_split(test_size=0.1, shuffle=True, seed=42, stratify_by_column="label")
hf_test  = hf_train['test']
hf_train = hf_train['train']

# Split the training set into training and validation sets
hf_train = hf_train.train_test_split(test_size=0.2, shuffle=True, seed=42, stratify_by_column="label")
# Rename just to make it easier to understand
hf_train = DatasetDict({
    'validation': hf_train['test'],
    'train': hf_train['train']
})
hf_validation = hf_train['validation']
hf_train = hf_train['train']

# Display the sizes of the splits
print(f"Train dataset size: {len(hf_train)}")
print(f"Validation dataset size: {len(hf_validation)}")
print(f"Test dataset size: {len(hf_test)}")

hf_train = hf_train.shuffle(seed=42).select(range(len(hf_train) // 10))
hf_validation = hf_validation.shuffle(seed=42).select(range(len(hf_validation) // 10))
hf_test = hf_test.shuffle(seed=42).select(range(len(hf_test) // 10))

print(f"Reduced Train dataset size: {len(hf_train)}")
print(f"Reduced Validation dataset size: {len(hf_validation)}")
print(f"Reduced Test dataset size: {len(hf_test)}")

"""

'\n# Split the emotion dataset into training, and test sets\nhf_train = dataset_emotion_split[\'train\'].train_test_split(test_size=0.1, shuffle=True, seed=42, stratify_by_column="label")\nhf_test  = hf_train[\'test\']\nhf_train = hf_train[\'train\']\n\n# Split the training set into training and validation sets\nhf_train = hf_train.train_test_split(test_size=0.2, shuffle=True, seed=42, stratify_by_column="label")\n# Rename just to make it easier to understand\nhf_train = DatasetDict({\n    \'validation\': hf_train[\'test\'],\n    \'train\': hf_train[\'train\']\n})\nhf_validation = hf_train[\'validation\']\nhf_train = hf_train[\'train\']\n\n# Display the sizes of the splits\nprint(f"Train dataset size: {len(hf_train)}")\nprint(f"Validation dataset size: {len(hf_validation)}")\nprint(f"Test dataset size: {len(hf_test)}")\n\nhf_train = hf_train.shuffle(seed=42).select(range(len(hf_train) // 10))\nhf_validation = hf_validation.shuffle(seed=42).select(range(len(hf_validation) // 10))\nhf_te

In [38]:

# Get class names from dataset features
class_names_train = dataset_emotion_split['train'].features['label'].names
class_names_val = dataset_emotion_split['validation'].features['label'].names
class_names_test = dataset_emotion_split['test'].features['label'].names

# Add new column mapping numeric labels to class names using Dataset.map
dataset_emotion_split['train'] = dataset_emotion_split['train'].map(
    lambda example: {'label_name': class_names_train[example['label']]}
)

dataset_emotion_split['validation'] = dataset_emotion_split['validation'].map(
    lambda example: {'label_name': class_names_val[example['label']]}
)

dataset_emotion_split['test'] = dataset_emotion_split['test'].map(
    lambda example: {'label_name': class_names_test[example['label']]}
)


"""
## Adding the class names to the datasets

## Train dataset
# Get class names from dataset features
class_names = hf_train.features['label'].names
#class_names = {0:'sadness', 1:'joy', 2:'love', 3:'anger', 4:'fear', 5:'surprise'}
# Create new column mapping numeric labels to class names
label_names = [class_names[label] for label in hf_train['label']]
# Add new column to dataset
hf_train = hf_train.add_column('label_name', label_names)

## Validation dataset
# Get class names from dataset features
class_names = hf_validation.features['label'].names
#class_names = {0:'sadness', 1:'joy', 2:'love', 3:'anger', 4:'fear', 5:'surprise'}
# Create new column mapping numeric labels to class names
label_names = [class_names[label] for label in hf_validation['label']]
# Add new column to dataset
hf_validation = hf_validation.add_column('label_name', label_names)

## Validation dataset
# Get class names from dataset features
class_names = hf_test.features['label'].names
#class_names = {0:'sadness', 1:'joy', 2:'love', 3:'anger', 4:'fear', 5:'surprise'}
# Create new column mapping numeric labels to class names
label_names = [class_names[label] for label in hf_test['label']]
# Add new column to dataset
hf_test = hf_test.add_column('label_name', label_names)


# Convert the datasets to a pandas DataFrame
df_train = pd.DataFrame(hf_train)
df_val = pd.DataFrame(hf_validation)
df_test = pd.DataFrame(hf_test)

"""
# Konverter det fulde datasæt til en pandas DataFrame
df_train = dataset_emotion_split['train'].to_pandas()
df_val = dataset_emotion_split['validation'].to_pandas()
df_test = dataset_emotion_split['test'].to_pandas()

# Kontroller for manglende værdier
mis_val_train = df_train.isnull().sum()
mis_val_val = df_val.isnull().sum()
mis_val_test = df_test.isnull().sum()

print('Number of missing values:')
print(mis_val_train,mis_val_val, mis_val_test)

# Sammenlign kolonnerne og find fælles værdier
print("FØRSTE TJEK FOR DUBLETTER I TRAIN OG VAL")
fælles_værdier = set(df_train['text']).intersection(set(df_val['text']))
print(len(fælles_værdier))
if fælles_værdier:
    print("Der er fælles værdier i train og val:", fælles_værdier)
else:
    print("Ingen fælles værdier i train og val.")


# Fjern rækker fra df_train, der også findes i df_val
#df_train_cleaned = df_train[~df_train['text'].isin(df_val['text'])]

# Fjern rækker fra df_val, der også findes i df_train
df_val_cleaned = df_val[~df_val['text'].isin(df_train['text'])]

#print(f"Rækker i df_train efter fjernelse af dubletter: {len(df_train_cleaned)}")
print(f"Rækker i df_val efter fjernelse af dubletter: {len(df_val_cleaned)}")


# Sammenlign kolonnerne og find fælles værdier
print("ANDET TJEK FOR DUBLETTER I TRAIN OG VAL")
fælles_værdier = set(df_train['text']).intersection(set(df_val_cleaned['text']))
print(len(fælles_værdier))
if fælles_værdier:
    print("Der er fælles værdier i train og val:", fælles_værdier)
else:
    print("Ingen fælles værdier i train og val.")





# Sammenlign kolonnerne og find fælles værdier i val og test
print("FØRSTE TJEK FOR DUBLETTER i VAL OG TEST")
fælles_værdier_val_test = set(df_val_cleaned['text']).intersection(set(df_test['text']))
print(len(fælles_værdier_val_test))
if fælles_værdier_val_test:
    print("Der er fælles værdier i validation og test:", fælles_værdier_val_test)
else:
    print("Ingen fælles værdier i validation og test.")


df_test_cleaned = df_test[~df_test['text'].isin(df_val_cleaned['text'])]


# Sammenlign kolonnerne og find fælles værdier i val og test
print("FØRSTE TJEK FOR DUBLETTER i VAL OG TEST")
fælles_værdier_val_test = set(df_val_cleaned['text']).intersection(set(df_test_cleaned['text']))
print(len(fælles_værdier_val_test))
if fælles_værdier_val_test:
    print("Der er fælles værdier i validation og test:", fælles_værdier_val_test)
else:
    print("Ingen fælles værdier i validation og test.")

Number of missing values:
text          0
label         0
label_name    0
dtype: int64 text          0
label         0
label_name    0
dtype: int64 text          0
label         0
label_name    0
dtype: int64
FØRSTE TJEK FOR DUBLETTER I TRAIN OG VAL
5
Der er fælles værdier i train og val: {'i feel so blessed to be a part of your days', 'i honestly feel at heart we should be faithful to each other if its yo girl', 'i miss our talks our cuddling our kissing and the feelings that you can only share with your beloved', 'i feel completely blessed to be a part of this group', 'i feel that being faithful isnt enough in your eyes'}
Rækker i df_val efter fjernelse af dubletter: 1995
ANDET TJEK FOR DUBLETTER I TRAIN OG VAL
0
Ingen fælles værdier i train og val.
FØRSTE TJEK FOR DUBLETTER i VAL OG TEST
3
Der er fælles værdier i validation og test: {'i don t always feel like i have amazing style and most days i choose comfort over anything else but there is one thing that i feel makes all the diffe

In [39]:
print(df_train.head())
print(df_val_cleaned.head())
print(df_test_cleaned.head())

                                                text  label label_name
0                            i didnt feel humiliated      0    sadness
1  i can go from feeling so hopeless to so damned...      0    sadness
2   im grabbing a minute to post i feel greedy wrong      3      anger
3  i am ever feeling nostalgic about the fireplac...      2       love
4                               i am feeling grouchy      3      anger
                                                text  label label_name
0  im feeling quite sad and sorry for myself but ...      0    sadness
1  i feel like i am still looking at a blank canv...      0    sadness
2                     i feel like a faithful servant      2       love
3                  i am just feeling cranky and blue      3      anger
4  i can have for a treat or if i am feeling festive      1        joy
                                                text  label label_name
0  im feeling rather rotten so im not very ambiti...      0    sadness
1     

In [40]:
df_train.to_pickle(f'{destination_path}/train.pkl')
df_val_cleaned.to_pickle(f'{destination_path}/validation.pkl')
df_test_cleaned.to_pickle(f'{destination_path}/test.pkl')

Sampling data to not be imbalanced

In [586]:
sadness = df_train[df_train['label'] == 0]
joy = df_train[df_train['label'] == 1]
love = df_train[df_train['label'] == 2]
anger = df_train[df_train['label'] == 3]
fear = df_train[df_train['label'] == 4]
surprise = df_train[df_train['label'] == 5]

number_rows = len(love)
print(number_rows)

df_sadness = df_train[df_train['label'] == 0].sample(n=number_rows)
df_joy = df_train[df_train['label'] == 1].sample(n=number_rows)
df_anger = df_train[df_train['label'] == 3].sample(n=number_rows)
df_fear = df_train[df_train['label'] == 4].sample(n=number_rows)
print(df_sadness.shape, df_joy.shape, df_anger.shape, df_fear.shape)

df_sample = pd.concat([df_sadness, df_joy, love, df_anger, df_fear, surprise], axis=0, ignore_index=True)
df_sample.shape

1304
(1304, 3) (1304, 3) (1304, 3) (1304, 3)


(7092, 3)

In [587]:
max_vocab_size = 10000

encoder = tf.keras.layers.TextVectorization(max_tokens=max_vocab_size)

#input_text = df_train['text'].to_list()

# Build a vocabulary of all string terms from tokens seen in the dataset
#encoder.adapt(input_text)

# Retrieves the computed vocabulary
#vocab = np.array(encoder.get_vocabulary()) 
#print(f'The reviews vocabulary: \n{vocab}\n')
#print(f'length of vocabulary: {len(vocab)}')

In [588]:
#nltk.download('stopwords')

In [589]:
#stop_words = set(stopwords.words('english'))

#def process_text(text):
#    tokenized_text = re.findall(r'\b\w+\b', text)
#    new_text = [word for word in tokenized_text if word.lower() not in stop_words]
#    return ' '.join(new_text)

#df_train['text_no_stop'] = df_train['text'].apply(process_text)


In [590]:
#input_text_new = df_train['text_no_stop'].to_list()

#encoder.adapt(input_text_new)

In [591]:
#vocab = np.array(encoder.get_vocabulary()) 
#print(f'The reviews vocabulary: \n{vocab}\n')
#print(f'length of vocabulary: {len(vocab)}')

In [592]:
lemmatizer = WordNetLemmatizer()

In [593]:
def process_text_lemma(text):
    tokenized_text = re.findall(r'\b\w+\b', text)
    lemmatized_text = [lemmatizer.lemmatize(word, pos = 'v') for word in tokenized_text]
    return ' '.join(lemmatized_text)


#df_train['text_lemma'] = df_train['text'].apply(process_text_lemma)
df_sample['text_lemma'] = df_sample['text'].apply(process_text_lemma)

In [594]:
#input_text_lemma = df_train['text_lemma'].to_list()
input_text_lemma = df_sample['text_lemma'].to_list()


encoder.adapt(input_text_lemma)

In [595]:
vocab = np.array(encoder.get_vocabulary()) 
print(f'The reviews vocabulary: \n{vocab}\n')
print(f'length of vocabulary: {len(vocab)}')

The reviews vocabulary: 
['' '[UNK]' 'i' ... 'aahhh' 'aac' 'aaaaaaand']

length of vocabulary: 8106


In [596]:
#def get_texts(data):
#    texts = data['text_lemma'].tolist()
#    labels = data['label'].tolist()
#    return texts, labels

#texts, labels = get_texts(df_train)

#tokenizer = Tokenizer(num_words=vocab_size)
#tokenizer.fit_on_texts(texts)

#X = tokenizer.texts_to_sequences(texts)
#X = pad_sequences(X, maxlen=max_length, padding='post')

Building an RNN model

In [601]:
def buildRNN (embedding_dimension, max_length, nb_input_features, nb_timesteps, nb_nodes, num_classes, vocab_size):
    model_rnn = tf.keras.models.Sequential()
    model_rnn.add(tf.keras.layers.Input(shape=(max_length,)))
    model_rnn.add(tf.keras.layers.Embedding(input_dim = vocab_size, 
                                            output_dim=embedding_dimension, 
                                            input_length = max_length, 
                                            name = "Embedding"))
    #model_rnn.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(nb_nodes,
    #                                                                 dropout=0.2,
    #                                                                 recurrent_dropout=0.2,
    #                                                                 return_sequences = True), 
    #                                                                 input_shape=(nb_timesteps, nb_input_features),
    #                                                                 name = 'LSTM_bi'))
    model_rnn.add(tf.keras.layers.GRU(units=nb_nodes, 
                                      return_sequences=True,
                                      recurrent_dropout=0.2,
                                      activation = 'relu',
                                      recurrent_activation = 'relu',
                                      #stateful=True,
                                      #recurrent_initializer='glorot_uniform',
                                      name = "GRU1"))


    model_rnn.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(units=nb_nodes, 
                                      return_sequences=True,
                                      #recurrent_dropout=0.2),
                                      #stateful=True,
                                      #recurrent_initializer='glorot_uniform',
                                      name = "GRU2")))
    
    model_rnn.add(tf.keras.layers.Dropout(0.2))

    model_rnn.add(tf.keras.layers.GRU(units=nb_nodes, 
                                      return_sequences=True,
                                      #recurrent_dropout=0.2,
                                      #stateful=True,
                                      #recurrent_initializer='glorot_uniform',
                                      name = "GRU3"))
    
    #model_rnn.add(tf.keras.layers.Dropout(0.2))

    model_rnn.add(tf.keras.layers.GRU(units=nb_nodes, 
                                      return_sequences=False,
                                      recurrent_dropout=0.2,
                                      #stateful=True,
                                      #recurrent_initializer='glorot_uniform',
                                      name = "GRU4"))
    
    #model_rnn.add(tf.keras.layers.Dropout(0.2))
    
    model_rnn.add(tf.keras.layers.Dense(num_classes, 
                                        activation='softmax',
                                        kernel_regularizer=tf.keras.regularizers.l2(0.01), 
                                        name = "Dense"))
    #rnn.summary()
    
    return model_rnn

In [602]:
def get_texts(data):
    texts = data['text_lemma'].tolist()
    labels = data['label'].tolist()
    return texts, labels

def get_texts_val(data):
    texts = data['text'].tolist()
    labels = data['label'].tolist()
    return texts, labels

In [604]:
embedding_dimension = [100]
max_length = [250]
num_classes = 6
vocab_size = len(vocab)

nb_input_features = [10000]
nb_timesteps = [250]
nb_nodes = [64]

epochs = [40]
batch_sizes = [32]

results = []


# Opret early stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',  # Hvad der skal overvåges (f.eks. 'val_loss' eller 'val_accuracy')
    patience=5,          # Antal epochs uden forbedring før stop
    restore_best_weights=True,  # Gendan vægtene fra den bedste epoch
    verbose=1            # Print en besked, når træningen stopper
)


for emb_dim in embedding_dimension:
    for m_len in max_length:
        for in_feat in nb_input_features:
            for t_step in nb_timesteps:
                for n in nb_nodes:
                    for epoch in epochs:
                        for batch in batch_sizes:
                            #optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
                            optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001)

                            #class_weights = {0: 1.0, 1: 1.0, 2: 4.0, 3:2.0, 4:3.0, 5:5.0}  # Vægt højere for mindre klasser
                            
                            
                            # Get X and y for training data
                            texts, labels = get_texts(df_sample)

                            tokenizer = Tokenizer(num_words=vocab_size)
                            tokenizer.fit_on_texts(texts)

                            X_sequences = tokenizer.texts_to_sequences(texts)
                            X = pad_sequences(X_sequences, maxlen=m_len, padding='post')

                            y = np.array(labels)

                            # Get X and y for validation data
                            texts_val, labels_val = get_texts_val(df_val)

                            #tokenizer = Tokenizer(num_words=vocab_size)
                            #tokenizer.fit_on_texts(texts_val)

                            X_val = tokenizer.texts_to_sequences(texts_val)
                            X_val = pad_sequences(X_val, maxlen=m_len, padding='post')

                            y_val = np.array(labels_val)

                            # Build the RNN
                            rnn = buildRNN(emb_dim, m_len, in_feat, t_step, n, num_classes, vocab_size)

                            rnn.compile(optimizer=optimizer,
                                            loss='sparse_categorical_crossentropy',
                                            metrics=['accuracy'],
                                            )
                            
                            history = rnn.fit(X, y,
                                              validation_data = (X_val, y_val),
                                              epochs = epoch,
                                              batch_size = batch,
                                              callbacks=[early_stopping],
                                              #class_weight=class_weights
                                              )
                            
                            sparse_categorical_crossentropy, accuracy = rnn.evaluate(X_val,y_val)

                            results.append([emb_dim, in_feat, t_step, n, epoch, sparse_categorical_crossentropy, accuracy])
                            
                            print(f'Embedding_dim: {emb_dim}, Input features: {in_feat}, Time steps: {t_step}, Nodes: {n}, Epochs: {epoch}, S_C_C-entropy: {sparse_categorical_crossentropy}, Accuracy: {accuracy}')
                            
                            #print(f'Activation = {i}: Test mse = {round(mse, 3)}, test mae = {round(mae, 3)}.')


                            # Plot the accuracy and loss history
                            fig = plt.figure(figsize=plt.figaspect(0.2))

                            ax = fig.add_subplot(1, 2, 1)
                            ax.plot(history.history['loss'], label='Training loss')
                            ax.plot(history.history['val_loss'], label='Validation loss')
                            plt.xlabel('Epoch')
                            plt.ylabel('Loss')
                            plt.legend()

                            ax = fig.add_subplot(1, 2, 2)
                            ax.plot(history.history['accuracy'], label='Training accuracy')
                            ax.plot(history.history['val_accuracy'], label='Validation accuracy')
                            plt.xlabel('Epoch')
                            plt.ylabel('Accuracy')
                            plt.legend()

                            plt.show()

Epoch 1/40
222/222 ━━━━━━━━━━━━━━━━━━━━ 40s 145ms/step - accuracy: 0.1800 - loss: 1.8797 - val_accuracy: 0.1060 - val_loss: 1.8323
Epoch 2/40
222/222 ━━━━━━━━━━━━━━━━━━━━ 32s 144ms/step - accuracy: 0.1881 - loss: 1.8527 - val_accuracy: 0.3520 - val_loss: 1.8072
Epoch 3/40
222/222 ━━━━━━━━━━━━━━━━━━━━ 32s 144ms/step - accuracy: 0.1709 - loss: 1.8478 - val_accuracy: 0.2750 - val_loss: 1.8016
Epoch 4/40
222/222 ━━━━━━━━━━━━━━━━━━━━ 36s 162ms/step - accuracy: 0.1818 - loss: 1.8273 - val_accuracy: 0.3520 - val_loss: 1.8014
Epoch 5/40
222/222 ━━━━━━━━━━━━━━━━━━━━ 39s 178ms/step - accuracy: 0.1883 - loss: 1.8247 - val_accuracy: 0.2750 - val_loss: 1.7863
Epoch 6/40
222/222 ━━━━━━━━━━━━━━━━━━━━ 32s 144ms/step - accuracy: 0.1806 - loss: 1.8130 - val_accuracy: 0.2750 - val_loss: 1.7854
Epoch 7/40
222/222 ━━━━━━━━━━━━━━━━━━━━ 32s 143ms/step - accuracy: 0.1912 - loss: 1.8021 - val_accuracy: 0.1375 - val_loss: 1.7789
Epoch 8/40
222/222 ━━━━━━━━━━━━━━━━━━━━ 32s 143ms/step - accuracy: 0.1769 - loss: 1

KeyboardInterrupt: 

In [ ]:
# evaluate the model
#loss, accuracy = model_rnn.evaluate(X_val, y_val)